In [1]:
import torchvision
import numpy as np
import TensionFlow as tf
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import torch
import random
import math
import importlib

In [2]:
importlib.reload(tf)

<module 'TensionFlow' from '/media/fakhir/SSD/AIExp/AutoGrad/TensionFlow/TensionFlow.py'>

In [3]:
train_data = torchvision.datasets.MNIST('data/', train=True)
test_data = torchvision.datasets.MNIST('data/', train=False)

In [4]:
train_data, test_data

(Dataset MNIST
     Number of datapoints: 60000
     Root location: data/
     Split: Train,
 Dataset MNIST
     Number of datapoints: 10000
     Root location: data/
     Split: Test)

**Forward Pass**

In [5]:
#defining model
f_in = 28*28
f_out = 10

# h_layer,b_layer = tf.LinearLayer(f_in, f_out)
# b_layer = tf.LinearLayer(1, f_out)

In [35]:
class Model:
    def __init__(self,n_layers,h_dim,f_in,f_out):
        self.in_w, self.in_b = tf.LinearLayer(f_in,h_dim)
        self.out_w, self.out_b = tf.LinearLayer(h_dim, f_out)
        self.layers = []
        self.biases = []
        for i in range(n_layers):
            w,b = tf.LinearLayer(h_dim,h_dim)
            self.layers.append(w)
            self.biases.append(b)
            
    
    def __call__(self, x):
        x = x @ self.in_w + self.in_b.broadcast(x.shape()[0])
        x = tf.ReLU(x)
        for w,b in zip(self.layers,self.biases):
            x = x @ w + b.broadcast(x.shape()[0])
            x = tf.ReLU(x)
        x = x @ self.out_w + self.out_b.broadcast(x.shape()[0])
        x = tf.Softmax(x)
        return x

    def update(self, lr):
        print(self.in_w.grad.sum())
        self.in_w -= self.in_w.grad * lr
        self.out_w -= self.out_w.grad * lr    
        self.out_b -= self.out_b.grad * lr    
        self.in_b -= self.in_b.grad * lr
        
        for w,b in zip(self.layers, self.biases):
            w -= w.grad * lr    
            b -= b.grad * lr                

In [42]:
lr = 1e-10

In [43]:
lossi = []
test_lossi = []

In [44]:
epochs = 1

In [45]:
batch_size = 1024

In [73]:
model = Model(0,50,f_in,f_out)

In [74]:
for e in (range(epochs)):
    t = tqdm(range(0,len(train_data), batch_size))
    for i in t:
#         print(train_data.data[i:i+batch_size].shape)
        current_batch_size = train_data.data[i:i+batch_size].shape[0]
        inp = train_data.data[i:i+batch_size].reshape(current_batch_size,28*28) / 255
        label = train_data.targets[i:i+batch_size]    
        
#         out = (tf.Neuron(inp.numpy()) @ h_layer) + b_layer.broadcast(current_batch_size)
#         out_soft = tf.Softmax(out)
        out_soft = model(tf.Neuron(inp.numpy()))
        oh_label = tf.one_hot(tf.Neuron(label.numpy()), f_out)
        
        loss = tf.CrossEntropy(out_soft,oh_label)
#         print(loss,out_soft)
        loss.backward()
#         h_layer.value -= h_layer.grad * lr
#         b_layer.value -= b_layer.grad.sum(0) * lr
        model.update(lr)
        lossi.append(loss.value[0][0])
        t.set_description(str(loss.value[0][0]))
        loss.backward_zero_grad()
        assert model.in_w.grad is None,f"{model.in_layer.grad}"
    
    for i in range(0,len(test_data), batch_size):
        current_batch_size = test_data.data[i:i+batch_size].shape[0]
        
        inp = test_data.data[i:i+batch_size].reshape(current_batch_size,28*28) / 255
        label = test_data.targets[i:i+batch_size]    
        
#         out = (tf.Neuron(inp.numpy()) @ h_layer) +  b_layer.broadcast(current_batch_size)
#         out_soft = tf.Softmax(out)
        out_soft = model(tf.Neuron(inp.numpy()))
        oh_label = tf.one_hot(tf.Neuron(label.numpy()), f_out)
        
        loss = tf.CrossEntropy(out_soft,oh_label)
        
        test_lossi.append(loss.value[0][0])

2350.99652117841:   5%|▌         | 3/59 [00:00<00:02, 22.32it/s] 

123708.47762636599
694144.0510072942
1620061.5741272569
8038710.157828562


2349.5481894562017:  14%|█▎        | 8/59 [00:00<00:02, 18.42it/s]

37845066.918154605
62295397.641501434
147633958.30964947
1713421003.5603473


2265.786892722658:  20%|██        | 12/59 [00:00<00:02, 16.20it/s] 

...

nan
nan
nan


nan:  98%|█████████▊| 58/59 [00:03<00:00, 13.04it/s]

nan
nan
nan


nan: 100%|██████████| 59/59 [00:03<00:00, 15.87it/s]


nan


In [ ]:
plt.plot(lossi[:])
x = list(range(0,len(lossi), math.ceil(len(lossi)/len(test_lossi))))
plt.plot(x, test_lossi[:len(x)])
plt.xlabel('steps')
plt.ylabel('loss')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
#accuracy
correct = 0
for i in range(0,len(test_data)//batch_size):
    current_batch_size = test_data.data[i:i+batch_size].shape[0]
    inp = test_data.data[i:i+batch_size].reshape(current_batch_size,28*28) / 255
    label = test_data.targets[i:i+batch_size]    

    out = (tf.Neuron(inp.numpy()) @ h_layer) +  b_layer.broadcast(current_batch_size)
    out_soft = tf.Softmax(out)
    correct += (out_soft.argmax(1) == label.numpy()).sum()


In [ ]:
correct/ len(test_data)

In [ ]:
#accuracy
correct = 0
for i in range(0,len(train_data)//batch_size):
    current_batch_size = train_data.data[i:i+batch_size].shape[0]
    inp = train_data.data[i:i+batch_size].reshape(current_batch_size,28*28) / 255
    label = train_data.targets[i:i+batch_size]    

    out = (tf.Neuron(inp.numpy()) @ h_layer) +  b_layer.broadcast(current_batch_size)
    out_soft = tf.Softmax(out)
    correct += (out_soft.argmax(1) == label.numpy()).sum()

In [ ]:
correct/ len(train_data)

In [ ]:
i = random.randint(0,len(test_data))
#inference
inp = test_data.data[i].reshape(1,28*28) / 255
label = test_data.targets[i]    

out = (tf.Neuron(inp.numpy()) @ h_layer) +  b_layer.broadcast(1)
out_soft = tf.Softmax(out)
print(f'Pred: {out_soft.argmax()}, label: {label}')
plt.imshow(test_data.data[i])
